In [1]:
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Model
from rdflib import Graph, Literal, Namespace, URIRef
from semantic_mpc_interface.namespaces import * 
from semantic_mpc_interface.utils import * 

In [2]:
bm = BuildingMOTIF("sqlite://")
model = Model.create(HPF)


In [3]:
tmp = Library.load(directory='dataclasses/templates/brick-templates')

/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/.venv/lib/python3.10/site-packages/pyshacl/extras/__init__.py:46: Warning: Extra "js" is not satisfied because requirement pyduktape2 is not installed.
  warn(Warning(f"Extra \"{extra_name}\" is not satisfied because requirement {req} is not installed."))


In [4]:
zone = tmp.get_template_by_name('hvac-zone')

In [5]:
zone_in = zone.inline_dependencies()

In [6]:
display(zone_in.all_parameters)

{'name',
 'space_name',
 'space_name-area_name',
 'space_name-area_unit',
 'space_name-area_value',
 'window_name',
 'window_name-area_name',
 'window_name-area_unit',
 'window_name-area_value',
 'window_name-azimuth_name',
 'window_name-azimuth_value',
 'window_name-tilt_name',
 'window_name-tilt_value'}

In [7]:
print(zone_in.body.serialize(format = 'ttl'))

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix ns1: <http://qudt.org/schema/qudt/> .

<urn:___param___#name> a brick:HVAC_Zone ;
    brick:hasPart <urn:___param___#space_name>,
        <urn:___param___#window_name> .

<urn:___param___#space_name> a brick:Space ;
    brick:area <urn:___param___#space_name-area_name> .

<urn:___param___#space_name-area_name> ns1:hasQuantityKind <http://qudt.org/vocab/quantitykind/Area> ;
    ns1:hasUnit <urn:___param___#space_name-area_unit> ;
    brick:value <urn:___param___#space_name-area_value> .

<urn:___param___#window_name> a brick:Window ;
    brick:area <urn:___param___#window_name-area_name> ;
    brick:azimuth <urn:___param___#window_name-azimuth_name> ;
    brick:tilt <urn:___param___#window_name-tilt_name> .

<urn:___param___#window_name-area_name> ns1:hasQuantityKind <http://qudt.org/vocab/quantitykind/Area> ;
    ns1:hasUnit <urn:___param___#window_name-area_unit> ;
    brick:value <urn:___param___#window_name-area_value>

In [8]:
def get_var_name(graph, node):
    pre, ns, local = graph.compute_qname(node)
    if PARAM == ns:
        q_n = f"?{local}".replace('-','_')
    else:
        q_n = convert_to_prefixed(node, graph).replace('-','_')
    return q_n

In [9]:
def make_where(graph):
    where = []
    for s,p,o in graph.triples((None,None,None)):
        # print(s,p,o)
        qs = get_var_name(graph, s)
        qo = get_var_name(graph, o)
        qp = convert_to_prefixed(p, graph).replace('-','_')
        where.append(f"{qs} {qp} {qo} .")
    return "\n".join(where)

In [10]:
def get_query(graph):
     where = make_where(graph)
     prefixes = get_prefixes(graph)
     query = f"""{prefixes}\nSELECT DISTINCT * WHERE {{ {where} }}"""
     return query

In [11]:
query = get_query(zone_in.body)

In [12]:
print(query)

PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX csvw: <http://www.w3.org/ns/csvw#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dcam: <http://purl.org/dc/dcam/>
PREFIX doap: <http://usefulinc.com/ns/doap#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX odrl: <http://www.w3.org/ns/odrl/2/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX prof: <http://www.w3.org/ns/dx/prof/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX schema: <https://schema.org/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX ssn: <http://www.w3.org/ns/ssn/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX vann: <http://purl.org/vocab/vann/>
PREFIX void: <http://rdfs.or

In [13]:
g = Graph()
g.parse('tutorial/bldg2.ttl')

<Graph identifier=N91bdd47fe8484839b9b2d471520d68ae (<class 'rdflib.graph.Graph'>)>

In [14]:
df = query_to_df(query, g)

In [15]:
class Value:
    unit: str
    value: float

In [16]:
df

,window_name_tilt_value,window_name_azimuth_value,window_name_area_name,window_name_area_value,window_name,space_name,space_name_area_unit,window_name_azimuth_name,name,space_name_area_name,window_name_area_unit,window_name_tilt_name,space_name_area_value
0,90.0,180.0,urn:hpflex/hpflex_demo#window_1_1_area,150.0,urn:hpflex/hpflex_demo#window_1_1,urn:hpflex/hpflex_demo#space_1_1,unit:FT2,urn:hpflex/hpflex_demo#window_1_1_azimuth,urn:hpflex/hpflex_demo#zone_1,urn:hpflex/hpflex_demo#space_1_1_area,unit:FT2,urn:hpflex/hpflex_demo#window_1_1_tilt,1500.0
1,90.0,180.0,urn:hpflex/hpflex_demo#window_2_1_area,150.0,urn:hpflex/hpflex_demo#window_2_1,urn:hpflex/hpflex_demo#space_2_1,unit:FT2,urn:hpflex/hpflex_demo#window_2_1_azimuth,urn:hpflex/hpflex_demo#zone_2,urn:hpflex/hpflex_demo#space_2_1_area,unit:FT2,urn:hpflex/hpflex_demo#window_2_1_tilt,1500.0
2,90.0,180.0,urn:hpflex/hpflex_demo#window_3_1_area,150.0,urn:hpflex/hpflex_demo#window_3_1,urn:hpflex/hpflex_demo#space_3_1,unit:FT2,urn:hpflex/hpflex_demo#window_3_1_azimuth,urn:hpflex/hpflex_demo#zone_3,urn:hpflex/hpflex_demo#space_3_1_area,unit:FT2,urn:hpflex/hpflex_demo#window_3_1_tilt,1500.0
3,90.0,180.0,urn:hpflex/hpflex_demo#window_4_1_area,150.0,urn:hpflex/hpflex_demo#window_4_1,urn:hpflex/hpflex_demo#space_4_1,unit:FT2,urn:hpflex/hpflex_demo#window_4_1_azimuth,urn:hpflex/hpflex_demo#zone_4,urn:hpflex/hpflex_demo#space_4_1_area,unit:FT2,urn:hpflex/hpflex_demo#window_4_1_tilt,1500.0
